<a href="https://colab.research.google.com/github/wyklq/AliOS-Things/blob/master/llmtune_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMTune: 4-Bit Finetuning of LLMs on a Consumer GPU
# Demo on Google Colab with T4 GPU (16G)
##1. Installation



### pip install -r requirements.txt (Note, pytorch is already installed)

In [1]:
!pip install sentencepiece==0.1.97
!pip install git+https://github.com/huggingface/transformers@9417c924af539be5f941c8a709a96b60dfe29eb3#egg=transformers
!pip install git+https://github.com/huggingface/peft@ff9a1edbfd2d405b86d50a2e5299cc1bbd49d887#egg=peft
!pip install datasets==2.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers (to revision 9417c924af539be5f941c8a709a96b60dfe29eb3) to /tmp/pip-install-fq48kgti/transformers_601ced2d639744a79c6f393385b813e2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-install-fq48kgti/transformers_601ced2d639744a79c6f393385b813e2
  Running command git rev-parse -q --verify 'sha^9417c924af539be5f941c8a709a96b60dfe29eb3'
  Running command git fetch -q https://github.com/huggingface/transformers 9417c924af539be5f941c8a709a96b60dfe29eb3
  Running command git checkout -q 9417c924af539be5f941c8a709a96b60dfe29eb3
  Resolved https://github.com/huggingface/transformers to commit 9417c924af539be5f941c

Clone llmtune source code and install.

In [3]:
!rm -rf llmtune && git clone https://github.com/wyklq/llmtune && cd llmtune && python setup.py install


Cloning into 'llmtune'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 74 (delta 26), reused 56 (delta 17), pack-reused 0
Unpacking objects: 100% (74/74), 124.72 KiB | 2.54 MiB/s, done.
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning

In [6]:
!cd llmtune && git pull

remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 94.96 KiB | 9.50 MiB/s, done.
From https://github.com/wyklq/llmtune
   8cbe43b..062c289  main       -> origin/main
Updating 8cbe43b..062c289
Fast-forward
 dataset_50cnalpaca.json | 1 +
 1 file changed, 1 insertion(+)
 create mode 100644 dataset_50cnalpaca.json


Download pre-trained and quantized model file and test if text generate with original llama-13b 4bit quantized model works

In [3]:
!wget https://huggingface.co/kuleshov/llama-13b-4bit/resolve/main/llama-13b-4bit.pt

--2023-05-20 06:33:28--  https://huggingface.co/kuleshov/llama-13b-4bit/resolve/main/llama-13b-4bit.pt
Resolving huggingface.co (huggingface.co)... 99.84.191.118, 99.84.191.66, 99.84.191.42, ...
Connecting to huggingface.co (huggingface.co)|99.84.191.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/5d/fe/5dfe252b94712f657ec0467f7dbddf7852ea12f46ffac06e5efd8b771608707b/56956757a1cf930a1881e7ade7bc4cdd243d88517b7d82cb06c01f11463caf38?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-13b-4bit.pt%3B+filename%3D%22llama-13b-4bit.pt%22%3B&Expires=1684823609&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzVkL2ZlLzVkZmUyNTJiOTQ3MTJmNjU3ZWMwNDY3ZjdkYmRkZjc4NTJlYTEyZjQ2ZmZhYzA2ZTVlZmQ4Yjc3MTYwODcwN2IvNTY5NTY3NTdhMWNmOTMwYTE4ODFlN2FkZTdiYzRjZGQyNDNkODg1MTdiN2Q4MmNiMDZjMDFmMTE0NjNjYWYzOD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaG

In [19]:
!wget https://huggingface.co/kuleshov/llama-7b-4bit/resolve/main/llama-7b-4bit.pt

--2023-05-20 06:53:54--  https://huggingface.co/kuleshov/llama-7b-4bit/resolve/main/llama-7b-4bit.pt
Resolving huggingface.co (huggingface.co)... 99.84.191.107, 99.84.191.42, 99.84.191.66, ...
Connecting to huggingface.co (huggingface.co)|99.84.191.107|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-05-20 06:53:54 ERROR 404: Not Found.



Download a dataset, and get the first 50 records.

In [5]:
!llmtune finetune --model llama-7b-4bit --weights llama-13b-4bit.pt --dataset llmtune/dataset_50cnalpaca.json --adapter llmtune

Traceback (most recent call last):
  File "/usr/local/bin/llmtune", line 33, in <module>
    sys.exit(load_entry_point('llmtune==0.1.0', 'console_scripts', 'llmtune')())
  File "/usr/local/lib/python3.10/dist-packages/llmtune-0.1.0-py3.10-linux-x86_64.egg/llmtune/run.py", line 101, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/llmtune-0.1.0-py3.10-linux-x86_64.egg/llmtune/run.py", line 142, in finetune
    from llmtune.executor import load_llm
  File "/usr/local/lib/python3.10/dist-packages/llmtune-0.1.0-py3.10-linux-x86_64.egg/llmtune/executor.py", line 7, in <module>
    from llmtune.engine.data import TrainTxt, TrainSAD, TrainGPT4All
  File "/usr/local/lib/python3.10/dist-packages/llmtune-0.1.0-py3.10-linux-x86_64.egg/llmtune/engine/data/__init__.py", line 1, in <module>
    from llmtune.engine.data.text import TrainTxt
  File "/usr/local/lib/python3.10/dist-packages/llmtune-0.1.0-py3.10-linux-x86_64.egg/llmtune/engine/data/text.py", line 3, in <module>